In [ ]:
#Read in data
import pandas as pd
import numpy as np

df = pd.read_csv('input.csv', header=0)
df.head()

In [ ]:
#Reduce DF to Relevant Variables
df_reduced = df[['CID', 'SALES','TRANS', 'QUANTITY', 'VISITS', 'CALLS', 'OFFERS'
                ]]
df_reduced.head()

In [ ]:
# Separating out the features
features = ['SALES','TRANS', 'QUANTITY', 'VISITS', 'CALLS', 'OFFERS'
           ]


In [ ]:
#Calculate Stats
df_reduced.mean()
df_reduced.std()

df_stats = pd.concat([df_reduced.mean(),df_reduced.std()], axis=1).round(2)
df_stats = df_stats.drop(['CID'])
df_stats.columns=['MEANS', 'STD DEV']
df_stats

In [ ]:
#Standardize the data
from sklearn.preprocessing import StandardScaler
# Separating out the features
x = df_reduced.loc[:, features].values
# Separating out the target
y = df_reduced.loc[:,['CID']].values
# Standardizing the features
x_std = StandardScaler().fit_transform(x)
x_std

In [ ]:
#Covariance Matrix

mean_vec = np.mean(x_std, axis=0)
cov_mat = (x_std - mean_vec).T.dot((x_std - mean_vec)) / (x_std.shape[0]-1)
print('Covariance matrix \n%s' %cov_mat)

cov_mat = np.cov(x_std.T)

eig_vals, eig_vecs = np.linalg.eig(cov_mat)

print('Eigenvectors \n%s' %eig_vecs)
print('\nEigenvalues \n%s' %eig_vals)

In [ ]:
#Correlation Matrix
cor_mat1 = np.corrcoef(x_std.T)

eig_vals, eig_vecs = np.linalg.eig(cor_mat1)

print('Eigenvectors \n%s' %eig_vecs)
print('\nEigenvalues \n%s' %eig_vals)

In [ ]:
#Singular Vector Decomposition
u,s,v = np.linalg.svd(x_std.T)
u

In [ ]:
#Make sure Eigenvectors have the same unit length 1
for ev in eig_vecs:
    np.testing.assert_array_almost_equal(1.0, np.linalg.norm(ev))
print('Everything ok!')

In [ ]:
# Make a list of (eigenvalue, eigenvector) tuples in descending order of explanatory power
eig_pairs = [(np.abs(eig_vals[i]), eig_vecs[:,i]) for i in range(len(eig_vals))]

# Sort the (eigenvalue, eigenvector) tuples from high to low
eig_pairs.sort()
eig_pairs.reverse()

In [ ]:
# Visually confirm that the list is correctly sorted by decreasing eigenvalues
print('Eigenvalues in descending order:')
for i in eig_pairs:
    print(i[0])
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
from sklearn.decomposition import PCA

pca = PCA().fit(x_std)
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance');

In [ ]:
# #Function to draw attention to significant loadings
# def color_significant_green(val):
#     """
#     Takes a scalar and returns a string with
#     the css property `'color: green'` for negative
#     strings, black otherwise.
#     """
#     color = 'green' if val <= -60 or val>=60 else 'black'
#     return 'color: %s' % color

def highlight_significant(val):
    """
    Takes a scalar and returns a string with
    the css property `'background-color: yellow'` for significant values, white otherwise.
    """
    color ='yellow' if val <= -60 or val>=60 else 'white'
    return 'background-color: %s' % color

In [ ]:
# Step 2. Compute loadings A. May skip if you don't need to interpret PCs anyhow.
# Loadings are eigenvectors normalized to respective eigenvalues: A value = V value * sqrt(L value)
# Loadings are the covariances between variables and components.

eig_vals_df = pd.DataFrame(np.array(eig_vals), columns=["Eigenvalues"])

In [ ]:
# Keep all vectors with values close to/greater than 1 
num_signif_vectors = np.count_nonzero(np.where(eig_vals_df['Eigenvalues']>=.9))+1

#PCs You Are Keeping
eig_vals_df2 = eig_vals_df.nlargest(num_signif_vectors, columns=["Eigenvalues"], keep='first')
# eig_vals_df2 = np.where(eig_vals_df['Eigenvalues']>=.9)

In [ ]:
eig_vals_pc1 = eig_vals_df2.iloc[0:1,:]
eig_vals_pc2 = eig_vals_df2.iloc[1:2,:]
eig_vals_pc3 = eig_vals_df2.iloc[2:3,:]
eig_vals_pc4 = eig_vals_df2.iloc[3:4,:]
eig_vals_pc5 = eig_vals_df2.iloc[4:5,:]
eig_vals_pc6 = eig_vals_df2.iloc[5:6,:]

eig_vecs_df = pd.DataFrame(np.array(eig_vecs))

eig_vecs_pc1 = eig_vecs_df.iloc[:,0:1]
eig_vecs_pc2 = eig_vecs_df.iloc[:,1:2]
eig_vecs_pc3 = eig_vecs_df.iloc[:,2:3]
eig_vecs_pc4 = eig_vecs_df.iloc[:,3:4]
eig_vecs_pc5 = eig_vecs_df.iloc[:,4:5]
eig_vecs_pc6 = eig_vecs_df.iloc[:,5:6]

#Calculate Loadings
factor_pattern_pc1 = eig_vecs_pc1.dot(np.sqrt(eig_vals_pc1))
factor_pattern_pc2 = eig_vecs_pc2.dot(np.sqrt(eig_vals_pc2))
factor_pattern_pc3 = eig_vecs_pc3.dot(np.sqrt(eig_vals_pc3))
factor_pattern_pc4 = eig_vecs_pc4.dot(np.sqrt(eig_vals_pc4))
factor_pattern_pc5 = eig_vecs_pc5.dot(np.sqrt(eig_vals_pc5))
factor_pattern_pc6 = eig_vecs_pc6.dot(np.sqrt(eig_vals_pc6))

In [ ]:
all_factors = pd.concat([factor_pattern_pc1,factor_pattern_pc2,factor_pattern_pc3,factor_pattern_pc4,factor_pattern_pc5,factor_pattern_pc6], axis=1).round(2)

all_factors.columns=['PC1', 'PC2', 'PC3', 'PC4', 'PC5', 'PC6']
all_factors.index=features
factor_pattern = all_factors*100
print("Significant Variables In Yellow")
factor_pattern.style.applymap(highlight_significant)
# all_factors.index=features

In [ ]:
# The communalities for the ith variable are computed by taking the sum of the squared loadings for that variable
all_factors_not_rounded = pd.concat([factor_pattern_pc1,factor_pattern_pc2,factor_pattern_pc3], axis=1)

communality_est = pd.DataFrame.sum(all_factors_not_rounded**2, axis=1)
communality_est = pd.DataFrame(communality_est)

communality_est.columns=["Final Communality Estimates"]
communality_est.index=features
print(communality_est)

total_communality_est = pd.DataFrame.sum(communality_est)
print(total_communality_est)

In [ ]:
#Orthogonal Transformation Matrix
def ortho_rotation(lam, method='varimax',gamma=None,
                   eps=1e-6, itermax=100):
    """
    Return orthogal rotation matrix
    TODO: - other types beyond 
    """
    if gamma == None:
        if (method == 'varimax'):
            gamma = 1.0
#         if (method == 'quartimax':
#             gamma = 0.0

    nrow, ncol = lam.shape
    R = np.eye(ncol)
    var = 0

    for i in range(itermax):
        lam_rot = np.dot(lam, R)
        tmp = np.diag(np.sum(lam_rot ** 2, axis=0)) / nrow * gamma
        u, s, v = np.linalg.svd(np.dot(lam.T, lam_rot ** 3 - np.dot(lam_rot, tmp)))
        R = np.dot(u, v)
        var_new = np.sum(s)
        if var_new < var * (1 + eps):
            break
        var = var_new

    return R

print("Orthogonal Transformation Matrix")
pd.DataFrame(ortho_rotation(factor_pattern)).round(5)